In [475]:
!pip install python-dotenv

In [2]:
!pip install opencv-python

In [3]:
import requests
import datetime
from urllib.parse import urlencode
from os import environ 
import base64
import urllib.request
import cv2
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import pandas as pd

In [4]:
from os.path import join
from dotenv import load_dotenv
#change from local 
env_path = '/Users/wesleyhouse/code/WesH0use/crate_scanner/.env'
load_dotenv(dotenv_path=env_path)

True

In [5]:
client_id = environ.get('SPOTIFY_CLIENT_ID')
client_secret = environ.get('SPOTIFY_CLIENT_SECRET')

client_id = client_id
client_secret = client_secret

In [525]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_track_by_id(self, _id):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/tracks"
        lookup_url = f"{endpoint}/{_id}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    

        
    def search(self, query=None, operator=None, operator_query=None, search_type='artist' ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)

        return self.base_search(query_params)

In [518]:
spotify = SpotifyAPI(client_id, client_secret)

In [593]:
spotify.get_track_by_id("5DlCyqLyX2AOVDTjjkDZ8x")["album"]

{'album_type': 'compilation',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2HS2wQTJXpA65XWOKlAVxk'},
   'href': 'https://api.spotify.com/v1/artists/2HS2wQTJXpA65XWOKlAVxk',
   'id': '2HS2wQTJXpA65XWOKlAVxk',
   'name': 'Mamie Smith',
   'type': 'artist',
   'uri': 'spotify:artist:2HS2wQTJXpA65XWOKlAVxk'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BY',
  'BZ',
  'CA',
  'CH',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JM',
  'JO',
  'JP',
  'KE',
  'KG',
  'KH',
  'KI',
  'KM',
  'KN',


In [521]:
song_df.head(10)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.643000,"""Screamin' Jay Hawkins""",0.852,150200,0.5170,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
1,0.993000,'Mamie Smith',0.647,163827,0.1860,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
2,0.000173,'Oscar Velazquez',0.730,422087,0.7980,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
3,0.295000,'Mixe',0.704,165224,0.7070,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920
4,0.996000,'Mamie Smith & Her Jazz Hounds',0.424,198627,0.2450,0,3HnrHGLE9u2MjHtdobfWl9,0.799000,5,0.2350,-11.470,1,Crazy Blues - 78rpm Version,9,1920,0.0397,103.870,0.4770,1920
5,0.992000,'Mamie Smith',0.782,195200,0.0573,0,5DlCyqLyX2AOVDTjjkDZ8x,0.000002,5,0.1760,-12.453,1,Don't You Advertise Your Man,5,1920,0.0592,85.652,0.4870,1920
6,0.996000,'Mamie Smith & Her Jazz Hounds',0.474,186173,0.2390,0,02FzJbHtqElixxCmrpSCUa,0.186000,9,0.1950,-9.712,1,Arkansas Blues,0,1920,0.0289,78.784,0.3660,1920
7,0.996000,'Francisco Canaro',0.469,146840,0.2380,0,02i59gYdjlhBmbbWhf8YuK,0.960000,8,0.1490,-18.717,1,La Chacarera - Remasterizado,0,1920-07-08,0.0741,130.060,0.6210,1920
8,0.006820,'Meetya',0.571,476304,0.7530,0,06NUxS2XL3efRh0bloxkHm,0.873000,8,0.0920,-6.943,1,Broken Puppet - Original Mix,0,1920-01-01,0.0446,126.993,0.1190,1920
9,0.952000,'Dorville',0.688,150067,0.2200,0,07jrRR1CUUoPb1FLfSy9Jh,0.000000,6,0.2620,-15.208,0,Oouin,0,1920,0.8450,82.024,0.4140,1920


In [8]:
spotify.search(query = 'Ok Computer Radiohead', search_type="album")

q=Ok+Computer+Radiohead&type=album


{'albums': {'href': 'https://api.spotify.com/v1/search?query=Ok+Computer+Radiohead&type=album&offset=0&limit=20',
  'items': [{'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
      'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
      'id': '4Z8W4fKeB5YxbusRsdQVPb',
      'name': 'Radiohead',
      'type': 'artist',
      'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}],
    'available_markets': ['AD',
     'AE',
     'AG',
     'AL',
     'AM',
     'AR',
     'AT',
     'AU',
     'AZ',
     'BA',
     'BB',
     'BD',
     'BE',
     'BF',
     'BG',
     'BH',
     'BI',
     'BN',
     'BO',
     'BR',
     'BS',
     'BT',
     'BW',
     'BY',
     'BZ',
     'CA',
     'CH',
     'CL',
     'CM',
     'CO',
     'CR',
     'CV',
     'CW',
     'CY',
     'CZ',
     'DE',
     'DK',
     'DM',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FJ'

In [589]:
%%time
test_album = spotify.search(query = 'Ok Computer Radiohead', search_type="album")

q=Ok+Computer+Radiohead&type=album
CPU times: user 24.8 ms, sys: 4.85 ms, total: 29.6 ms
Wall time: 262 ms


In [592]:
get_album_image(test_album)

KeyError: 'images'

In [49]:
album_df = test_df["artists"]

In [50]:
album_df = pd.DataFrame(album_df)

In [559]:
def clean_artists_col(artists):
    cleaned_text = artists.replace('[','').replace(']','')
    return cleaned_text

def get_album_image(album):
    # Use API search query to extract the album image url
    if album and len(album['images'])>0:
        return album['images'][0]['url']
    else:
        return None

def get_album_name(album):
    # Use API search query to extract the album name
    if album:
        return album['name']
    else:
        return None
    
def get_album_id(album):
    # Use API search query to extract the unique Spotify album id
    if album:
        return album['id']
    else:
        return None
    
def get_album(artist):
    album_search = spotify.search(query = artist, search_type="album")['albums']['items']
    if len(album_search)>0:
        return album_search[0]
    else:
        return None
    
def get_album_by_track_id(track_id):
    album_search = spotify.get_track_by_id(track_id)["album"]
    if len(album_search)>0:
        return album_search['id']
    else:
        return None

In [600]:
test_album = get_album("Radiohead Ok Computer")

q=Radiohead+Ok+Computer&type=album


In [572]:
song_df_test = song_df[0:20].copy

In [529]:
song_df.tail(10)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
990,0.996,'Louis Armstrong & His Hot Five',0.568,173760,0.1560,0,1k4xhU1UoTGs2lYsA7xq4J,0.734000,11,0.209,-16.906,1,I'm Gonna Gitcha,1,1925,0.3170,186.689,0.8000,1925
991,0.984,'Dranem',0.671,149133,0.3660,0,1usQRVBEb4UMMlUdsPPfOV,0.000000,7,0.115,-13.625,1,Raymonde,0,1925,0.0618,109.651,0.8940,1925
992,0.992,'Georgius',0.541,196867,0.3780,0,1xSOWXoINyYVF46ab8sKpb,0.000102,7,0.131,-15.167,1,Hortensia Ou La Fille Du Jazz Band,0,1925,0.0777,140.419,0.7330,1925
993,0.816,'Georges Thill',0.298,217261,0.0436,0,1zPmz3rh854RXIho4RYtWi,0.007290,0,0.111,-19.321,1,"Faust, Acte 3, Scène Et Duo Du Jardin, Fin (4è...",0,1925-01-01,0.1930,76.279,0.0543,1925
994,0.996,"'Francisco Canaro', 'Azucena Maizani'",0.563,174667,0.2420,0,230yKx4iJH3uW33JkEqP4g,0.912000,0,0.398,-12.030,1,Silbando - Remasterizado,0,1925-08-02,0.1470,113.549,0.4170,1925
995,0.967,'Ethel Waters',0.580,187133,0.2250,0,23furE1azaxq02V6iHZHMt,0.000000,0,0.118,-16.784,1,Come Up And See Me Sometime,2,1925,0.3190,96.869,0.5020,1925
996,0.996,'Louis Armstrong & His Hot Five',0.686,181600,0.1270,0,29735iAeYhkpYnVAURWpSa,0.372000,7,0.112,-17.711,1,Heebie Jeebies,2,1925,0.0818,156.359,0.5920,1925
997,0.992,'Yvonne George',0.553,170173,0.0903,0,2AbVQuMyvNhUZEsNAQzp8V,0.000000,3,0.259,-14.881,1,J'ai Pas Su Y Faire,0,1925,0.1010,77.049,0.3790,1925
998,0.957,'Tommy Dorsey',0.775,224573,0.2350,0,2F8CnNSU7pExOSXzsgf2vz,0.000000,0,0.955,-11.981,1,Oh! Look At Me Now,2,1925,0.1080,120.288,0.6010,1925
999,0.993,'Francisco Canaro',0.443,171533,0.2650,0,2HHvjG9sSrs5uVeity8AvB,0.959000,1,0.141,-17.091,1,Virgencita del Talar - Remasterizado,0,1925-07-12,0.4600,69.038,0.3150,1925


In [532]:
get_album_by_track_id("4xB5WxQgapIwtmxTolX7Uk")

'3hWomTJKGucGKv6EPpiHK9'

In [225]:
second_album_df = pd.read_csv('data.csv', skiprows=15000, nrows=1000,header=None)

In [228]:
second_album_df['artists'] = second_album_df[1]

In [230]:
second_album_df = second_album_df['artists']

In [232]:
second_album_df = pd.DataFrame(second_album_df)

In [238]:
second_album_df['artists'] = second_album_df['artists'].apply(clean_artists_col)

In [256]:
second_album_df['album_cover'] = second_album_df['artists'].apply(get_album_image)

q=%27Real+McCoy%27&type=album
q=%27DJ+Quik%27&type=album
q=%27Chalino+Sanchez%27&type=album
q=%27No+Doubt%27&type=album
q=%27Trisha+Yearwood%27&type=album
q=%27Andrea+Bocelli%27&type=album
q=%27Slaughter%27&type=album
q=%27Nightcrawlers%27%2C+%27MK%27&type=album
q=%27ODB%27&type=album
q=%27Bruce+Springsteen%27&type=album
q=%27LL+Cool+J%27&type=album
q=%27Alanis+Morissette%27&type=album
q=%27GZA%27%2C+%27RZA%27%2C+%27Ghostface+Killah%27%2C+%27Killah+Priest%27&type=album
q=%27Foo+Fighters%27&type=album
q=%27Grand+Puba%27&type=album
q=%27Bone+Thugs-N-Harmony%27&type=album
q=%27Tom+T.+Hall%27&type=album
q=%27Van+Halen%27&type=album
q=%27Shakira%27&type=album
q=%27Pulp%27&type=album
q=%27Collective+Soul%27&type=album
q=%272Pac%27&type=album
q=%27Shakira%27&type=album
q=%27Shakira%27&type=album
q=%27Cher%27&type=album
q=%27Faith+Hill%27&type=album
q=%27Ace+of+Base%27&type=album
q=%27Misfits%27%2C+%27Glenn+Danzig%27&type=album
q=%27Monica%27&type=album
q=%27Miguel+Y+Miguel%27&type=album
q=%27

q=%27Donell+Jones%27&type=album
q=%27Sublime%27&type=album
q=%27Maxwell%27&type=album
q=%27Weezer%27&type=album
q=%27Mint+Condition%27&type=album
q=%27Sheryl+Crow%27&type=album
q=%27Mazzy+Star%27&type=album
q=%27Beck%27&type=album
q=%272Pac%27%2C+%27Storm%27%2C+%27Mutah%27%2C+%22Michel%27le%22&type=album
q=%27Blind+Melon%27&type=album
q=%27Better+Than+Ezra%27&type=album
q=%27Bloodhound+Gang%27&type=album
q=%27The+Suicide+Machines%27&type=album
q=%27TOOL%27&type=album
q=%27Backstreet+Boys%27&type=album
q=%27Van+Halen%27&type=album
q=%27Los+Tigres+Del+Norte%27&type=album
q=%27Fugees%27&type=album
q=%27Redman%27&type=album
q=%27A+Tribe+Called+Quest%27&type=album
q=%27Nas%27&type=album
q=%27Misfits%27&type=album
q=%27Foo+Fighters%27&type=album
q=%27Third+Eye+Blind%27&type=album
q=%27John+Denver%27&type=album
q=%27The+Notorious+B.I.G.%27&type=album
q=%27Green+Day%27&type=album
q=%27The+Verve%27&type=album
q=%27The+Notorious+B.I.G.%27%2C+%27Mase%27%2C+%27Diddy%27&type=album
q=%27George+Strai

q=%27Yvonne+Elliman%27&type=album
q=%27Usher%27&type=album
q=%27Mary+J.+Blige%27&type=album
q=%27Dizzy+Gillespie%27&type=album
q=%27Bobby+Helms%27&type=album
q=%27%2ANSYNC%27&type=album
q=%27George+Michael%27&type=album
q=%27The+Goo+Goo+Dolls%27&type=album
q=%27Rob+Zombie%27&type=album
q=%27The+Offspring%27&type=album
q=%27Korn%27&type=album
q=%27Vince+Guaraldi+Trio%27&type=album
q=%27JUVENILE%27%2C+%27Lil+Wayne%27%2C+%27Mannie+Fresh%27&type=album
q=%27Ms.+Lauryn+Hill%27&type=album
q=%27Barenaked+Ladies%27&type=album
q=%272Pac%27%2C+%27Outlawz%27&type=album
q=%27Burl+Ives%27&type=album
q=%27New+Radicals%27&type=album
q=%27Eve+6%27&type=album
q=%27Donny+Osmond%27%2C+%27Chorus+-+Mulan%27&type=album
q=%27The+Chicks%27&type=album
q=%27The+Goo+Goo+Dolls%27&type=album
q=%27T-Dre%27%2C+%27Delux%27%2C+%27Cali+Life+Style%27&type=album
q=%22Lil%27+Troy%22&type=album
q=%27Godsmack%27&type=album
q=%27Big+Pun%27%2C+%27Joe%27&type=album
q=%27DMX%27&type=album
q=%272Pac%27&type=album
q=%272Pac%27%2C+

q=%27Dr.+Dre%27%2C+%27Snoop+Dogg%27&type=album
q=%27Limp+Bizkit%27&type=album
q=%27Britney+Spears%27&type=album
q=%27MF+DOOM%27%2C+%27DJ+Cucumber+Slice%27&type=album
q=%27Tal+Bachman%27&type=album
q=%27Foo+Fighters%27&type=album
q=%27Ginuwine%27&type=album
q=%27George+Strait%27&type=album
q=%27Lou+Bega%27&type=album
q=%27Incubus%27&type=album
q=%27Korn%27&type=album
q=%27Kool+%26+The+Gang%27&type=album
q=%22Destiny%27s+Child%22&type=album
q=%27The+Chicks%27&type=album
q=%27Creed%27&type=album
q=%27Eminem%27&type=album
q=%27Creed%27&type=album
q=%27blink-182%27&type=album
q=%27Crazy+Town%27&type=album
q=%27Rage+Against+The+Machine%27&type=album
q=%27Dr.+Dre%27%2C+%27Hittman%27%2C+%27Six-Two%27%2C+%27Nate+Dogg%27%2C+%27Kurupt%27&type=album
q=%27Christina+Aguilera%27&type=album
q=%27DMX%27&type=album
q=%27Lonestar%27&type=album
q=%27Powerman+5000%27&type=album
q=%27Mos+Def%27&type=album
q=%27Tim+McGraw%27&type=album
q=%27Staind%27&type=album
q=%27Los+Socios+Del+Ritmo%27&type=album
q=%27Ke

q=%27RichaadEB%27&type=album
q=%27Uncle+Kracker%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27UB40%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27Limp+Bizkit%27&type=album
q=%27David+Bowie%27&type=album
q=%27The+Beatles%27&type=album
q=%27Selena%27&type=album
q=%27Bon+Jovi%27&type=album
q=%27Godhead%27&type=album
q=%27Christina+Aguilera%27&type=album
q=%27Billy+Joel%27&type=album
q=%27Linkin+Park%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27Outkast%27&type=album
q=%27Billy+Joel%27&type=album
q=%27Gloria+Gaynor%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27Shaggy%27%2C+%27Rayvon%27&type=album
q=%27Mogwai%27&type=album
q=%27Eminem%27&type=album
q=%27Billy+Joel%27&type=album
q=%27Lee+Ann+Womack%27&type=album
q=%27Burning+Spear%27&type=album
q=%27Deftones%27&type=album
q=%27Foo+Fighters%27&type=album
q=%27Wheatus%27&type=album
q=%27David+Bowie+%26+The+Spiders+From+Mars%27&type=album
q=%27Christina+Aguilera%27&type=album
q=%27Billy+Joel%27&type=al

In [259]:
second_album_df["album_name"] = second_album_df['artists'].apply(get_album_name)

q=%27Real+McCoy%27&type=album
q=%27DJ+Quik%27&type=album
q=%27Chalino+Sanchez%27&type=album
q=%27No+Doubt%27&type=album
q=%27Trisha+Yearwood%27&type=album
q=%27Andrea+Bocelli%27&type=album
q=%27Slaughter%27&type=album
q=%27Nightcrawlers%27%2C+%27MK%27&type=album
q=%27ODB%27&type=album
q=%27Bruce+Springsteen%27&type=album
q=%27LL+Cool+J%27&type=album
q=%27Alanis+Morissette%27&type=album
q=%27GZA%27%2C+%27RZA%27%2C+%27Ghostface+Killah%27%2C+%27Killah+Priest%27&type=album
q=%27Foo+Fighters%27&type=album
q=%27Grand+Puba%27&type=album
q=%27Bone+Thugs-N-Harmony%27&type=album
q=%27Tom+T.+Hall%27&type=album
q=%27Van+Halen%27&type=album
q=%27Shakira%27&type=album
q=%27Pulp%27&type=album
q=%27Collective+Soul%27&type=album
q=%272Pac%27&type=album
q=%27Shakira%27&type=album
q=%27Shakira%27&type=album
q=%27Cher%27&type=album
q=%27Faith+Hill%27&type=album
q=%27Ace+of+Base%27&type=album
q=%27Misfits%27%2C+%27Glenn+Danzig%27&type=album
q=%27Monica%27&type=album
q=%27Miguel+Y+Miguel%27&type=album
q=%27

q=%27Donell+Jones%27&type=album
q=%27Sublime%27&type=album
q=%27Maxwell%27&type=album
q=%27Weezer%27&type=album
q=%27Mint+Condition%27&type=album
q=%27Sheryl+Crow%27&type=album
q=%27Mazzy+Star%27&type=album
q=%27Beck%27&type=album
q=%272Pac%27%2C+%27Storm%27%2C+%27Mutah%27%2C+%22Michel%27le%22&type=album
q=%27Blind+Melon%27&type=album
q=%27Better+Than+Ezra%27&type=album
q=%27Bloodhound+Gang%27&type=album
q=%27The+Suicide+Machines%27&type=album
q=%27TOOL%27&type=album
q=%27Backstreet+Boys%27&type=album
q=%27Van+Halen%27&type=album
q=%27Los+Tigres+Del+Norte%27&type=album
q=%27Fugees%27&type=album
q=%27Redman%27&type=album
q=%27A+Tribe+Called+Quest%27&type=album
q=%27Nas%27&type=album
q=%27Misfits%27&type=album
q=%27Foo+Fighters%27&type=album
q=%27Third+Eye+Blind%27&type=album
q=%27John+Denver%27&type=album
q=%27The+Notorious+B.I.G.%27&type=album
q=%27Green+Day%27&type=album
q=%27The+Verve%27&type=album
q=%27The+Notorious+B.I.G.%27%2C+%27Mase%27%2C+%27Diddy%27&type=album
q=%27George+Strai

q=%27Yvonne+Elliman%27&type=album
q=%27Usher%27&type=album
q=%27Mary+J.+Blige%27&type=album
q=%27Dizzy+Gillespie%27&type=album
q=%27Bobby+Helms%27&type=album
q=%27%2ANSYNC%27&type=album
q=%27George+Michael%27&type=album
q=%27The+Goo+Goo+Dolls%27&type=album
q=%27Rob+Zombie%27&type=album
q=%27The+Offspring%27&type=album
q=%27Korn%27&type=album
q=%27Vince+Guaraldi+Trio%27&type=album
q=%27JUVENILE%27%2C+%27Lil+Wayne%27%2C+%27Mannie+Fresh%27&type=album
q=%27Ms.+Lauryn+Hill%27&type=album
q=%27Barenaked+Ladies%27&type=album
q=%272Pac%27%2C+%27Outlawz%27&type=album
q=%27Burl+Ives%27&type=album
q=%27New+Radicals%27&type=album
q=%27Eve+6%27&type=album
q=%27Donny+Osmond%27%2C+%27Chorus+-+Mulan%27&type=album
q=%27The+Chicks%27&type=album
q=%27The+Goo+Goo+Dolls%27&type=album
q=%27T-Dre%27%2C+%27Delux%27%2C+%27Cali+Life+Style%27&type=album
q=%22Lil%27+Troy%22&type=album
q=%27Godsmack%27&type=album
q=%27Big+Pun%27%2C+%27Joe%27&type=album
q=%27DMX%27&type=album
q=%272Pac%27&type=album
q=%272Pac%27%2C+

q=%27Dr.+Dre%27%2C+%27Snoop+Dogg%27&type=album
q=%27Limp+Bizkit%27&type=album
q=%27Britney+Spears%27&type=album
q=%27MF+DOOM%27%2C+%27DJ+Cucumber+Slice%27&type=album
q=%27Tal+Bachman%27&type=album
q=%27Foo+Fighters%27&type=album
q=%27Ginuwine%27&type=album
q=%27George+Strait%27&type=album
q=%27Lou+Bega%27&type=album
q=%27Incubus%27&type=album
q=%27Korn%27&type=album
q=%27Kool+%26+The+Gang%27&type=album
q=%22Destiny%27s+Child%22&type=album
q=%27The+Chicks%27&type=album
q=%27Creed%27&type=album
q=%27Eminem%27&type=album
q=%27Creed%27&type=album
q=%27blink-182%27&type=album
q=%27Crazy+Town%27&type=album
q=%27Rage+Against+The+Machine%27&type=album
q=%27Dr.+Dre%27%2C+%27Hittman%27%2C+%27Six-Two%27%2C+%27Nate+Dogg%27%2C+%27Kurupt%27&type=album
q=%27Christina+Aguilera%27&type=album
q=%27DMX%27&type=album
q=%27Lonestar%27&type=album
q=%27Powerman+5000%27&type=album
q=%27Mos+Def%27&type=album
q=%27Tim+McGraw%27&type=album
q=%27Staind%27&type=album
q=%27Los+Socios+Del+Ritmo%27&type=album
q=%27Ke

q=%27RichaadEB%27&type=album
q=%27Uncle+Kracker%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27UB40%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27Limp+Bizkit%27&type=album
q=%27David+Bowie%27&type=album
q=%27The+Beatles%27&type=album
q=%27Selena%27&type=album
q=%27Bon+Jovi%27&type=album
q=%27Godhead%27&type=album
q=%27Christina+Aguilera%27&type=album
q=%27Billy+Joel%27&type=album
q=%27Linkin+Park%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27Outkast%27&type=album
q=%27Billy+Joel%27&type=album
q=%27Gloria+Gaynor%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27Shaggy%27%2C+%27Rayvon%27&type=album
q=%27Mogwai%27&type=album
q=%27Eminem%27&type=album
q=%27Billy+Joel%27&type=album
q=%27Lee+Ann+Womack%27&type=album
q=%27Burning+Spear%27&type=album
q=%27Deftones%27&type=album
q=%27Foo+Fighters%27&type=album
q=%27Wheatus%27&type=album
q=%27David+Bowie+%26+The+Spiders+From+Mars%27&type=album
q=%27Christina+Aguilera%27&type=album
q=%27Billy+Joel%27&type=al

In [261]:
second_album_df["album_id"] = second_album_df['artists'].apply(get_album_id)

q=%27Real+McCoy%27&type=album
q=%27DJ+Quik%27&type=album
q=%27Chalino+Sanchez%27&type=album
q=%27No+Doubt%27&type=album
q=%27Trisha+Yearwood%27&type=album
q=%27Andrea+Bocelli%27&type=album
q=%27Slaughter%27&type=album
q=%27Nightcrawlers%27%2C+%27MK%27&type=album
q=%27ODB%27&type=album
q=%27Bruce+Springsteen%27&type=album
q=%27LL+Cool+J%27&type=album
q=%27Alanis+Morissette%27&type=album
q=%27GZA%27%2C+%27RZA%27%2C+%27Ghostface+Killah%27%2C+%27Killah+Priest%27&type=album
q=%27Foo+Fighters%27&type=album
q=%27Grand+Puba%27&type=album
q=%27Bone+Thugs-N-Harmony%27&type=album
q=%27Tom+T.+Hall%27&type=album
q=%27Van+Halen%27&type=album
q=%27Shakira%27&type=album
q=%27Pulp%27&type=album
q=%27Collective+Soul%27&type=album
q=%272Pac%27&type=album
q=%27Shakira%27&type=album
q=%27Shakira%27&type=album
q=%27Cher%27&type=album
q=%27Faith+Hill%27&type=album
q=%27Ace+of+Base%27&type=album
q=%27Misfits%27%2C+%27Glenn+Danzig%27&type=album
q=%27Monica%27&type=album
q=%27Miguel+Y+Miguel%27&type=album
q=%27

q=%27Donell+Jones%27&type=album
q=%27Sublime%27&type=album
q=%27Maxwell%27&type=album
q=%27Weezer%27&type=album
q=%27Mint+Condition%27&type=album
q=%27Sheryl+Crow%27&type=album
q=%27Mazzy+Star%27&type=album
q=%27Beck%27&type=album
q=%272Pac%27%2C+%27Storm%27%2C+%27Mutah%27%2C+%22Michel%27le%22&type=album
q=%27Blind+Melon%27&type=album
q=%27Better+Than+Ezra%27&type=album
q=%27Bloodhound+Gang%27&type=album
q=%27The+Suicide+Machines%27&type=album
q=%27TOOL%27&type=album
q=%27Backstreet+Boys%27&type=album
q=%27Van+Halen%27&type=album
q=%27Los+Tigres+Del+Norte%27&type=album
q=%27Fugees%27&type=album
q=%27Redman%27&type=album
q=%27A+Tribe+Called+Quest%27&type=album
q=%27Nas%27&type=album
q=%27Misfits%27&type=album
q=%27Foo+Fighters%27&type=album
q=%27Third+Eye+Blind%27&type=album
q=%27John+Denver%27&type=album
q=%27The+Notorious+B.I.G.%27&type=album
q=%27Green+Day%27&type=album
q=%27The+Verve%27&type=album
q=%27The+Notorious+B.I.G.%27%2C+%27Mase%27%2C+%27Diddy%27&type=album
q=%27George+Strai

q=%27Yvonne+Elliman%27&type=album
q=%27Usher%27&type=album
q=%27Mary+J.+Blige%27&type=album
q=%27Dizzy+Gillespie%27&type=album
q=%27Bobby+Helms%27&type=album
q=%27%2ANSYNC%27&type=album
q=%27George+Michael%27&type=album
q=%27The+Goo+Goo+Dolls%27&type=album
q=%27Rob+Zombie%27&type=album
q=%27The+Offspring%27&type=album
q=%27Korn%27&type=album
q=%27Vince+Guaraldi+Trio%27&type=album
q=%27JUVENILE%27%2C+%27Lil+Wayne%27%2C+%27Mannie+Fresh%27&type=album
q=%27Ms.+Lauryn+Hill%27&type=album
q=%27Barenaked+Ladies%27&type=album
q=%272Pac%27%2C+%27Outlawz%27&type=album
q=%27Burl+Ives%27&type=album
q=%27New+Radicals%27&type=album
q=%27Eve+6%27&type=album
q=%27Donny+Osmond%27%2C+%27Chorus+-+Mulan%27&type=album
q=%27The+Chicks%27&type=album
q=%27The+Goo+Goo+Dolls%27&type=album
q=%27T-Dre%27%2C+%27Delux%27%2C+%27Cali+Life+Style%27&type=album
q=%22Lil%27+Troy%22&type=album
q=%27Godsmack%27&type=album
q=%27Big+Pun%27%2C+%27Joe%27&type=album
q=%27DMX%27&type=album
q=%272Pac%27&type=album
q=%272Pac%27%2C+

q=%27Dr.+Dre%27%2C+%27Snoop+Dogg%27&type=album
q=%27Limp+Bizkit%27&type=album
q=%27Britney+Spears%27&type=album
q=%27MF+DOOM%27%2C+%27DJ+Cucumber+Slice%27&type=album
q=%27Tal+Bachman%27&type=album
q=%27Foo+Fighters%27&type=album
q=%27Ginuwine%27&type=album
q=%27George+Strait%27&type=album
q=%27Lou+Bega%27&type=album
q=%27Incubus%27&type=album
q=%27Korn%27&type=album
q=%27Kool+%26+The+Gang%27&type=album
q=%22Destiny%27s+Child%22&type=album
q=%27The+Chicks%27&type=album
q=%27Creed%27&type=album
q=%27Eminem%27&type=album
q=%27Creed%27&type=album
q=%27blink-182%27&type=album
q=%27Crazy+Town%27&type=album
q=%27Rage+Against+The+Machine%27&type=album
q=%27Dr.+Dre%27%2C+%27Hittman%27%2C+%27Six-Two%27%2C+%27Nate+Dogg%27%2C+%27Kurupt%27&type=album
q=%27Christina+Aguilera%27&type=album
q=%27DMX%27&type=album
q=%27Lonestar%27&type=album
q=%27Powerman+5000%27&type=album
q=%27Mos+Def%27&type=album
q=%27Tim+McGraw%27&type=album
q=%27Staind%27&type=album
q=%27Los+Socios+Del+Ritmo%27&type=album
q=%27Ke

q=%27RichaadEB%27&type=album
q=%27Uncle+Kracker%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27UB40%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27Limp+Bizkit%27&type=album
q=%27David+Bowie%27&type=album
q=%27The+Beatles%27&type=album
q=%27Selena%27&type=album
q=%27Bon+Jovi%27&type=album
q=%27Godhead%27&type=album
q=%27Christina+Aguilera%27&type=album
q=%27Billy+Joel%27&type=album
q=%27Linkin+Park%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27Outkast%27&type=album
q=%27Billy+Joel%27&type=album
q=%27Gloria+Gaynor%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27Shaggy%27%2C+%27Rayvon%27&type=album
q=%27Mogwai%27&type=album
q=%27Eminem%27&type=album
q=%27Billy+Joel%27&type=album
q=%27Lee+Ann+Womack%27&type=album
q=%27Burning+Spear%27&type=album
q=%27Deftones%27&type=album
q=%27Foo+Fighters%27&type=album
q=%27Wheatus%27&type=album
q=%27David+Bowie+%26+The+Spiders+From+Mars%27&type=album
q=%27Christina+Aguilera%27&type=album
q=%27Billy+Joel%27&type=al

In [268]:
album_dataset = pd.concat([album_df, second_album_df], ignore_index=True)

In [271]:
third_album_df = pd.read_csv('data.csv', skiprows=11000, nrows=1000,header=None)

In [273]:
third_album_df['artists'] =  third_album_df[1]

In [282]:
third_album_df = pd.DataFrame(third_album_df)

In [285]:
third_album_df['artists'] = third_album_df['artists'].apply(clean_artists_col)

In [328]:
third_album_df = third_album_df.drop('artists')

In [330]:
third_album_df['album'] = third_album_df['artists'].map(get_album)

q=%27Ben+E.+King%27&type=album
q=%27Fleetwood+Mac%27&type=album
q=%27Queen%27&type=album
q=%27Little+Feat%27&type=album
q=%27%E3%82%BB%E3%82%AD%E3%83%88%E3%82%AA%E3%83%BB%E3%82%B7%E3%82%B2%E3%82%AA%27&type=album
q=%27Jerry+Jeff+Walker%27&type=album
q=%27Janis+Ian%27&type=album
q=%27El+Gran+Combo+De+Puerto+Rico%27&type=album
q=%22The+O%27Jays%22&type=album
q=%27Jeff+Beck%27&type=album
q=%27Aerosmith%27&type=album
q=%27Elton+John%27&type=album
q=%27Fleetwood+Mac%27&type=album
q=%27The+Who%27&type=album
q=%27Queen%27&type=album
q=%27John+Denver%27&type=album
q=%27Bruce+Springsteen%27&type=album
q=%27Led+Zeppelin%27&type=album
q=%27Mtume%27&type=album
q=%27Led+Zeppelin%27&type=album
q=%27War%27&type=album
q=%27Burning+Spear%27&type=album
q=%27Brian+Eno%27&type=album
q=%27Don+Williams%27&type=album
q=%27Led+Zeppelin%27&type=album
q=%27Gordon+Lightfoot%27&type=album
q=%27James+Taylor%27&type=album
q=%27Bruce+Springsteen%27&type=album
q=%27Barry+White%27&type=album
q=%27Grateful+Dead%27&type=

q=%27Joni+Mitchell%27&type=album
q=%27Marty+Robbins%27%2C+%27Ray+Conniff%27&type=album
q=%27Bad+Company%27&type=album
q=%27The+Modern+Lovers%27&type=album
q=%27England+Dan+%26+John+Ford+Coley%27&type=album
q=%27Billy+Joel%27&type=album
q=%27Boz+Scaggs%27&type=album
q=%27KISS%27&type=album
q=%27Henry+Gross%27&type=album
q=%27Ramon+Ayala+Y+Sus+Bravos+Del+Norte%27&type=album
q=%27Bob+Dylan%27&type=album
q=%27Eagles%27&type=album
q=%27Fleetwood+Mac%27&type=album
q=%27Fleetwood+Mac%27&type=album
q=%27Electric+Light+Orchestra%27&type=album
q=%27Fleetwood+Mac%27&type=album
q=%27Heart%27&type=album
q=%27Bill+Withers%27&type=album
q=%27Queen%27&type=album
q=%27Billy+Joel%27&type=album
q=%27Bob+Marley+%26+The+Wailers%27&type=album
q=%27Commodores%27&type=album
q=%27Kansas%27&type=album
q=%27Jimmy+Buffett%27&type=album
q=%27Eric+Clapton%27&type=album
q=%27Supertramp%27&type=album
q=%27Cheap+Trick%27&type=album
q=%27Styx%27&type=album
q=%27Fleetwood+Mac%27&type=album
q=%27David+Bowie%27&type=album

q=%27Gerry+Rafferty%27&type=album
q=%27Waylon+Jennings%27%2C+%27Willie+Nelson%27&type=album
q=%27Rick+James%27&type=album
q=%27Van+Halen%27&type=album
q=%27Van+Halen%27&type=album
q=%27The+Cars%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27Village+People%27&type=album
q=%27Journey%27&type=album
q=%27Foreigner%27&type=album
q=%27Gerry+Rafferty%27&type=album
q=%27Warren+Zevon%27&type=album
q=%27Ramones%27&type=album
q=%27Bob+Seger%27&type=album
q=%27REO+Speedwagon%27&type=album
q=%27Van+Halen%27&type=album
q=%27The+Jacksons%27&type=album
q=%27John+Travolta%27%2C+%27Olivia+Newton-John%27&type=album
q=%27The+Cars%27&type=album
q=%27Don+Williams%27&type=album
q=%27Journey%27&type=album
q=%27Grateful+Dead%27&type=album
q=%27Blondie%27&type=album
q=%27Jimmy+Buffett%27&type=album
q=%27Ambrosia%27&type=album
q=%27Bob+Seger%27&type=album
q=%27Delegation%27&type=album
q=%27REO+Speedwagon%27&type=album
q=%27Carpenters%27&type=album
q=%27Kate+Bush%27&type=album
q=%27Chaka+Khan%27&type=al

q=%27Pink+Floyd%27&type=album
q=%27Kenny+Rogers%27&type=album
q=%27Electric+Light+Orchestra%27&type=album
q=%27Donnie+%26+Joe+Emerson%27&type=album
q=%27The+Oak+Ridge+Boys%27&type=album
q=%27CHIC%27&type=album
q=%27Steve+Forbert%27&type=album
q=%27ABBA%27&type=album
q=%27AC%2FDC%27&type=album
q=%27XTC%27&type=album
q=%27Dan+Fogelberg%27&type=album
q=%27Bad+Company%27&type=album
q=%27Prince%27&type=album
q=%27Joe+Jackson%27&type=album
q=%27Sister+Sledge%27&type=album
q=%27Tom+Petty+and+the+Heartbreakers%27&type=album
q=%27Kermit%27&type=album
q=%27Nick+Lowe%27&type=album
q=%27Kenny+Loggins%27&type=album
q=%27Donna+Summer%27&type=album
q=%27The+Cars%27&type=album
q=%27Hank+Williams%2C+Jr.%27%2C+%27Waylon+Jennings%27&type=album
q=%27Randy+VanWarmer%27&type=album
q=%27John+Denver%27%2C+%27The+Muppets%27&type=album
q=%27AC%2FDC%27&type=album
q=%27Dionne+Warwick%27&type=album
q=%27Bob+Marley+%26+The+Wailers%27&type=album
q=%27Donna+Summer%27&type=album
q=%27Eagles%27&type=album
q=%27Michael+

q=%27Van+Halen%27&type=album
q=%27Bob+Seger%27&type=album
q=%27U2%27&type=album
q=%27Alabama%27&type=album
q=%27ABBA%27&type=album
q=%27ABBA%27&type=album
q=%27The+Rolling+Stones%27&type=album
q=%27The+Jam%27&type=album
q=%27Talking+Heads%27&type=album
q=%27Ramones%27&type=album
q=%27Bow+Wow+Wow%27&type=album
q=%27Miguel+R%C3%ADos%27&type=album
q=%27John+Lennon%27&type=album
q=%27Neil+Diamond%27&type=album
q=%27The+J.+Geils+Band%27&type=album


In [356]:
third_album_df['album_cover'] = third_album_df['album'].map(get_album_image)

In [360]:
third_album_df['album_name'] = third_album_df['album'].map(get_album_name)

In [362]:
third_album_df['album_id'] = third_album_df['album'].map(get_album_id)

In [365]:
del third_album_df['album']

In [367]:
album_dataset = pd.concat([album_dataset, third_album_df], ignore_index=True)

In [369]:
album_dataset

,artists,album_cover,album_name,album_id
0,'Mamie Smith',https://i.scdn.co/image/ab67616d000048510057f5...,Mamie Smith Vol. 1 (1920-1921),5R4HBYPhpEXGLXeyjWFbmT
1,"""Screamin' Jay Hawkins""",https://i.scdn.co/image/ab67616d00004851c8a136...,At Home with Screamin' Jay Hawkins,2lKtE0pl4RtMD2bUwKzdcz
2,'Mamie Smith',https://i.scdn.co/image/ab67616d000048510057f5...,Mamie Smith Vol. 1 (1920-1921),5R4HBYPhpEXGLXeyjWFbmT
3,'Oscar Velazquez',https://i.scdn.co/image/ab67616d00004851fd2df2...,This Is True House Music,63wFVCPZxdaFL9WKXbIQ5m
4,'Mixe',https://i.scdn.co/image/ab67616d00004851832333...,Mixed Emotions,2iSs4IZZHQ5lrFIbUqC1NX
...,...,...,...,...
2995,'Bow Wow Wow',https://i.scdn.co/image/ab67616d000048510779a8...,We Are The '80s,3gTAnUMTCOtCHp0zb4gdSF
2996,'Miguel Ríos',https://i.scdn.co/image/ab67616d00004851c8aaaf...,Rocanrol Bumerang,1YIw799VStxgysT27cUGwp
2997,'John Lennon',https://i.scdn.co/image/ab67616d00004851995815...,Imagine,0xzaemKucrJpYhyl7TltAk
2998,'Neil Diamond',https://i.scdn.co/image/ab67616d000048512803d0...,Sweet Caroline,6RfgcwsOUlWkGNAd6zjjYd


In [370]:
best_albums_dataset = pd.read_csv("best_albums.csv")

In [372]:
best_albums_dataset.head(50)

,Rank,Title,Band,Country,Year,Compilation,Live,YearRank,DecadeRank,OverallRank,RankScore,AvgRating,NumRatings,AlbumID,Notes,Buy
0,0,"=HYPERLINK(""""https://www.besteveralbums.com/ov...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,OK Computer,Radiohead,uk,1997.0,No,No,1.0,1.0,1.0,103160.0,91.853363,5023.0,227.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."
2,2,The Dark Side Of The Moon,Pink Floyd,uk,1973.0,No,No,1.0,1.0,2.0,95111.0,92.282722,4767.0,203.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."
3,3,Abbey Road,The Beatles,uk,1969.0,No,No,1.0,1.0,3.0,82871.0,91.642052,4111.0,24.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."
4,4,Revolver,The Beatles,uk,1966.0,No,No,1.0,2.0,4.0,75445.0,91.192093,4091.0,28.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."
5,5,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,uk,1967.0,No,No,1.0,3.0,5.0,69902.0,90.482063,4116.0,30.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."
6,6,Kid A,Radiohead,uk,2000.0,No,No,1.0,1.0,6.0,69845.0,89.347794,3979.0,1002.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."
7,7,In Rainbows,Radiohead,uk,2007.0,No,No,1.0,2.0,7.0,65426.0,89.409790,3476.0,1706.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."
8,8,Wish You Were Here,Pink Floyd,uk,1975.0,No,No,1.0,2.0,8.0,64085.0,91.229210,3576.0,358.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."
9,9,The Rise And Fall Of Ziggy Stardust And The Sp...,David Bowie,uk,1972.0,No,No,1.0,3.0,9.0,63795.0,90.109940,3111.0,74.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."


In [373]:
best_albums_dataset['artists'] = best_albums_dataset['Band']

In [375]:
best_albums_dataset['album_name'] = best_albums_dataset['Title']

In [381]:
del best_albums_dataset["Compilation"]

In [382]:
best_albums_dataset

,Rank,Title,Band,Live,YearRank,DecadeRank,OverallRank,RankScore,AvgRating,NumRatings,AlbumID,Notes,Buy,artists,album_name
0,0,"=HYPERLINK(""""https://www.besteveralbums.com/ov...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/ov..."
1,1,OK Computer,Radiohead,No,1.0,1.0,1.0,103160.0,91.853363,5023.0,227.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu...",Radiohead,OK Computer
2,2,The Dark Side Of The Moon,Pink Floyd,No,1.0,1.0,2.0,95111.0,92.282722,4767.0,203.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu...",Pink Floyd,The Dark Side Of The Moon
3,3,Abbey Road,The Beatles,No,1.0,1.0,3.0,82871.0,91.642052,4111.0,24.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu...",The Beatles,Abbey Road
4,4,Revolver,The Beatles,No,1.0,2.0,4.0,75445.0,91.192093,4091.0,28.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu...",The Beatles,Revolver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9997,9997,Brian Davison's Every Which Way,Brian Davison,No,163.0,1523.0,9997.0,113.0,75.425629,7.0,93067.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu...",Brian Davison,Brian Davison's Every Which Way
9998,9998,Westworld,Theatre Of Hate,No,157.0,1471.0,9998.0,113.0,72.305031,17.0,20714.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu...",Theatre Of Hate,Westworld
9999,9999,Three Snakes And One Charm,The Black Crowes,No,228.0,1814.0,9999.0,113.0,69.658363,42.0,20522.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu...",The Black Crowes,Three Snakes And One Charm
10000,10000,Damage Control,Mat Zo,No,261.0,2155.0,10000.0,113.0,75.482391,5.0,62956.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu...",Mat Zo,Damage Control


In [387]:
best_albums_dataset = best_albums_dataset.drop(['DecadeRank', 'OverallRank', 'RankScore', "AvgRating", "NumRatings", "AlbumID", "Notes", "Buy"], axis = 1) 

In [391]:
best_albums_dataset = best_albums_dataset.drop(["Rank", "Live", "YearRank"], axis=1)

In [398]:
best_albums_dataset = best_albums_dataset.drop(["Title", "Band"], axis=1)

In [409]:
best_albums_dataset["album_artist"] = best_albums_dataset["artists"] + " " + best_albums_dataset["album_name"] 

In [411]:
best_albums_dataset["album"] = best_albums_dataset['album_artist'].map(get_album)

q=nan&type=album
q=Radiohead+OK+Computer&type=album
q=Pink+Floyd+The+Dark+Side+Of+The+Moon&type=album
q=The+Beatles+Abbey+Road&type=album
q=The+Beatles+Revolver&type=album
q=The+Beatles+Sgt.+Pepper%27s+Lonely+Hearts+Club+Band&type=album
q=Radiohead+Kid+A&type=album
q=Radiohead+In+Rainbows&type=album
q=Pink+Floyd+Wish+You+Were+Here&type=album
q=David+Bowie+The+Rise+And+Fall+Of+Ziggy+Stardust+And+The+Spiders+From+Mars&type=album
q=The+Velvet+Underground+%26+Nico+The+Velvet+Underground+%26+Nico&type=album
q=Led+Zeppelin+Untitled+%28Led+Zeppelin+IV%29&type=album
q=Arcade+Fire+Funeral&type=album
q=The+Beach+Boys+Pet+Sounds&type=album
q=The+Beatles+The+Beatles+%28The+White+Album%29&type=album
q=Nirvana+Nevermind&type=album
q=Neutral+Milk+Hotel+In+The+Aeroplane+Over+The+Sea&type=album
q=Pixies+Doolittle&type=album
q=The+Clash+London+Calling&type=album
q=The+Smiths+The+Queen+Is+Dead&type=album
q=Radiohead+The+Bends&type=album
q=My+Bloody+Valentine+Loveless&type=album
q=The+Strokes+Is+This+It&t

q=Bruce+Springsteen+Born+In+The+U.S.A.&type=album
q=Soundgarden+Superunknown&type=album
q=The+Doors+L.A.+Woman&type=album
q=Arctic+Monkeys+AM&type=album
q=Blur+Parklife&type=album
q=Violent+Femmes+Violent+Femmes&type=album
q=Green+Day+Dookie&type=album
q=Arcade+Fire+Reflektor&type=album
q=Pavement+Slanted+And+Enchanted&type=album
q=The+Stooges+Fun+House&type=album
q=Prince+Sign+%27O%27+The+Times&type=album
q=Bj%C3%B6rk+Vespertine&type=album
q=Frank+Zappa+Hot+Rats&type=album
q=Kanye+West+Yeezus&type=album
q=Beastie+Boys+Paul%27s+Boutique&type=album
q=Steely+Dan+Aja&type=album
q=Coldplay+Parachutes&type=album
q=Beach+House+Bloom&type=album
q=Creedence+Clearwater+Revival+Cosmo%27s+Factory&type=album
q=Queens+Of+The+Stone+Age+...Like+Clockwork&type=album
q=Ramones+Ramones&type=album
q=Neil+Young+On+The+Beach&type=album
q=Death+Grips+The+Money+Store&type=album
q=Genesis+The+Lamb+Lies+Down+On+Broadway&type=album
q=Car+Seat+Headrest+Teens+Of+Denial&type=album
q=Captain+Beefheart+%26+His+Magic

q=OutKast+Stankonia&type=album
q=Can+Future+Days&type=album
q=Tom+Waits+Swordfishtrombones&type=album
q=Sly+%26+The+Family+Stone+There%27s+A+Riot+Goin%27+On&type=album
q=XTC+Skylarking&type=album
q=Bright+Eyes+I%27m+Wide+Awake%2C+It%27s+Morning&type=album
q=Vampire+Weekend+Contra&type=album
q=Run+The+Jewels+Run+The+Jewels+2&type=album
q=Kraftwerk+Die+Mensch%C2%B7Maschine+%5BThe+Man-Machine%5D&type=album
q=R.E.M.+Document&type=album
q=Supertramp+Breakfast+In+America&type=album
q=Elvis+Presley+Elvis+Presley&type=album
q=Talking+Heads+Speaking+In+Tongues&type=album
q=Kids+See+Ghosts+Kids+See+Ghosts&type=album
q=Bruce+Springsteen+The+River&type=album
q=The+Kinks+Arthur+%28Or+The+Decline+And+Fall+Of+The+British+Empire%29&type=album
q=Bob+Dylan+Desire&type=album
q=Franz+Ferdinand+Franz+Ferdinand&type=album
q=The+National+Alligator&type=album
q=Frank+Sinatra+In+The+Wee+Small+Hours&type=album
q=Red+Hot+Chili+Peppers+By+The+Way&type=album
q=Have+A+Nice+Life+Deathconsciousness&type=album
q=Swans

q=Meat+Loaf+Bat+Out+Of+Hell&type=album
q=Milton+Nascimento+%2F+L%C3%B4+Borges+Clube+Da+Esquina&type=album
q=Joni+Mitchell+Court+And+Spark&type=album
q=Gorillaz+Gorillaz&type=album
q=Freddie+Gibbs+%26+Madlib+Pi%C3%B1ata&type=album
q=MGMT+Congratulations&type=album
q=Talking+Heads+More+Songs+About+Buildings+And+Food&type=album
q=Steve+Reich+Music+For+18+Musicians&type=album
q=Art+Blakey+%26+The+Jazz+Messengers+Art+Blakey+And+The+Jazz+Messengers&type=album
q=MF+DOOM+Mm..+Food&type=album
q=D%27Angelo+Voodoo&type=album
q=Kate+Bush+The+Kick+Inside&type=album
q=Death+Cab+For+Cutie+Plans&type=album
q=The+Mothers+Of+Invention+We%27re+Only+In+It+For+The+Money&type=album
q=Jamie+xx+In+Colour&type=album
q=St.+Vincent+Strange+Mercy&type=album
q=Creedence+Clearwater+Revival+Willy+And+The+Poor+Boys&type=album
q=Rush+Hemispheres&type=album
q=Julia+Holter+Have+You+In+My+Wilderness&type=album
q=Big+Star+3rd&type=album
q=My+Bloody+Valentine+m+b+v&type=album
q=Live+Throwing+Copper&type=album
q=Alice+In+Ch

q=Sonic+Youth+Dirty&type=album
q=Rainbow+Rising&type=album
q=Mumford+%26+Sons+Sigh+No+More&type=album
q=Sun+Kil+Moon+Ghosts+Of+The+Great+Highway&type=album
q=Beastie+Boys+Ill+Communication&type=album
q=Daft+Punk+Homework&type=album
q=U2+All+That+You+Can%27t+Leave+Behind&type=album
q=Florence+%2B+The+Machine+Ceremonials&type=album
q=Devo+Q%3A+Are+We+Not+Men%3F+A%3A+We+Are+Devo%21&type=album
q=Miles+Davis+Birth+Of+The+Cool&type=album
q=Depeche+Mode+Black+Celebration&type=album
q=The+Allman+Brothers+Band+Eat+A+Peach&type=album
q=The+Crickets+The+%26quot%3BChirping%26quot%3B+Crickets&type=album
q=Curtis+Mayfield+Curtis&type=album
q=The+Offspring+Smash&type=album
q=My+Morning+Jacket+Z&type=album
q=The+Beatles+Beatles+For+Sale&type=album
q=Guns+N%27+Roses+Use+Your+Illusion+II&type=album
q=Mogwai+Mogwai+Young+Team&type=album
q=Howlin%27+Wolf+Moanin%27+In+The+Moonlight&type=album
q=13th+Floor+Elevators+The+Psychedelic+Sounds+Of+The+13th+Floor+Elevators&type=album
q=PJ+Harvey+Rid+Of+Me&type=alb

...

q=Poni+Hoax+Images+Of+Sigrid&type=album
q=Pat+Metheny+Group+Travels&type=album
q=Frank+Iero+And+The+Future+Violents+Barriers&type=album
q=%E1%88%99%E1%88%8B%E1%89%B1+%E1%8A%A0%E1%88%B5%E1%89%B3%E1%8C%A5%E1%89%84+%5BMulatu+Astatke%5D+%C3%89thiopiques+4%3A+Ethio+Jazz+%26+Musique+Instrumentale+1969-1974&type=album
q=Rickie+Lee+Jones+The+Magazine&type=album
q=Roky+Erickson+Roky+Erickson+And+The+Aliens+%285+Symbols%29&type=album
q=Jack+Stauber+HiLo&type=album
q=Beyonc%C3%A9+Homecoming%3A+The+Live+Album&type=album
q=Miles+Davis+Tutu&type=album
q=Bruce+Cockburn+In+The+Falling+Dark&type=album
q=DJ+Screw+June+27th&type=album
q=Silver+Jews+Bright+Flight&type=album
q=Hazel+Scott+Her+Second+Album+Of+Piano+Solos+With+Drums+Acc.&type=album
q=Anja+Garbarek+The+Road+Is+Just+A+Surface&type=album
q=Aimee+Mann+%40%23%25%26%2A%21+Smilers&type=album
q=Clearlake+Lido&type=album
q=B.B.+King+%26+Eric+Clapton+Riding+With+The+King&type=album
q=The+Ananda+Shankar+Experience+And+State+Of+Bengal+Walking+On&type=al

q=Bessie+Smith+Empress+Of+The+Blues&type=album
q=The+Cure+Mixed+Up&type=album
q=Baroness+Yellow+%26+Green&type=album
q=Berlin+Pleasure+Victim&type=album
q=Wilson+Pickett+In+The+Midnight+Hour&type=album
q=Hiss+Golden+Messenger+Hallelujah+Anyhow&type=album
q=Laurel+Halo+Raw+Silk+Uncut+Wood&type=album
q=Kristin+Hersh+Sunny+Border+Blue&type=album
q=Anekdoten+From+Within&type=album
q=Parkway+Drive+Deep+Blue&type=album
q=The+Wonder+Stuff+Hup&type=album
q=The+Dentists+Some+People+Are+On+The+Pitch+They+Think+It%27s+All+Over+It+Is+Now&type=album
q=Sweet+Sweet+Fanny+Adams&type=album
q=Bob+Dylan+Hard+Rain&type=album
q=Suburban+Lawns+Suburban+Lawns&type=album
q=Eddy+Grant+Killer+On+The+Rampage&type=album
q=Miami+Sound+Machine+Primitive+Love&type=album
q=Hombres+G+Hombres+G&type=album
q=Stevie+Wonder+For+Once+In+My+Life&type=album
q=Slade+Slade+In+Flame&type=album
q=J.I.D+DiCaprio+2&type=album
q=Carcass+Swansong&type=album
q=Richard+Thompson+You%3F+Me%3F+Us%3F&type=album
q=Fuck+Buttons+Street+Horrr

q=Ka+Descendants+Of+Cain&type=album
q=Rosanne+Cash+The+River+And+The+Thread&type=album
q=Bob+Dylan+%26+The+Band+The+Bootleg+Series+Vol.+11%3A+The+Basement+Tapes+Complete&type=album
q=Bonfire+Fireworks&type=album
q=The+Rapture+In+The+Grace+Of+Your+Love&type=album
q=Frog+Eyes+Paul%27s+Tomb%3A+A+Triumph&type=album
q=Jah+Lion+Colombia+Colly&type=album
q=Fennesz+Black+Sea&type=album
q=Pat+Metheny+Group+Letter+From+Home&type=album
q=Jorja+Smith+Lost+%26+Found&type=album
q=Hammock+Departure+Songs&type=album
q=Fela+Anikulapo+Kuti+%26+Egypt+80+Beasts+Of+No+Nation&type=album
q=%C2%B5-Ziq+Royal+Astronomy&type=album
q=Univers+Zero+Ceux+Du+Dehors&type=album
q=Lazerhawk+Redline&type=album
q=National+Health+Of+Queues+And+Cures&type=album
q=Eric+Clapton+Pilgrim&type=album
q=All+Time+Low+Don%27t+Panic&type=album
q=Calexico+Hot+Rail&type=album
q=Inoyama+Land+Danzindan-Pojidon&type=album
q=Fettes+Brot+Am+Wasser+Gebaut&type=album
q=Junip+Fields&type=album
q=Alvin+Lucier+I+Am+Sitting+In+A+Room&type=album
q

q=Graveyard+Graveyard&type=album
q=Willie+Dixon+The+Chess+Box&type=album
q=The+6ths+Wasps%27+Nests&type=album
q=Fields+Of+The+Nephilim+Elizium&type=album
q=Meshuggah+Nothing&type=album
q=The+Dillinger+Escape+Plan+Dissociation&type=album
q=Tom+Z%C3%A9+Grande+Liquida%C3%A7%C3%A3o&type=album
q=Anne+Clark+Changing+Places&type=album
q=Bruno+Walter+%2F+New+York+Philharmonic+Mahler%3A+Symphony+No.+5&type=album
q=Various+Artists+Platinum+Breakz&type=album
q=Sylvester+Step+II&type=album
q=Beach+Slang+The+Things+We+Do+To+Find+People+Who+Feel+Like+Us&type=album
q=Beirut+Lon+Gisland&type=album
q=The+Waterboys+An+Appointment+With+Mr.+Yeats&type=album
q=Wild+Beasts+Limbo%2C+Panto&type=album
q=Frank+Zappa+L%C3%A4ther&type=album
q=Herbert+von+Karajan+%2F+Berlin+Philharmonic+Orchestra+Beethoven%3A+Symphony+No.+9+%26quot%3BChoral%26quot%3B&type=album
q=The+West+Coast+Pop+Art+Experimental+Band+Vol.+2+%28Breaking+Through%29&type=album
q=Igor+Stravinsky+%2F+The+Philharmonic+Symphony+Orchestra+Of+New+York+S

q=Robert+Plant+Band+Of+Joy&type=album
q=OFF%21+First+Four+EPs&type=album
q=Brian+Davison+Brian+Davison%27s+Every+Which+Way&type=album
q=Theatre+Of+Hate+Westworld&type=album
q=The+Black+Crowes+Three+Snakes+And+One+Charm&type=album
q=Mat+Zo+Damage+Control&type=album
q=nan&type=album


,artists,album_name,album_artist,album
0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/ov...",NaN,"{'album_type': 'album', 'artists': [{'external..."
1,Radiohead,OK Computer,Radiohead OK Computer,"{'album_type': 'album', 'artists': [{'external..."
2,Pink Floyd,The Dark Side Of The Moon,Pink Floyd The Dark Side Of The Moon,"{'album_type': 'album', 'artists': [{'external..."
3,The Beatles,Abbey Road,The Beatles Abbey Road,"{'album_type': 'album', 'artists': [{'external..."
4,The Beatles,Revolver,The Beatles Revolver,"{'album_type': 'album', 'artists': [{'external..."
...,...,...,...,...
9997,Brian Davison,Brian Davison's Every Which Way,Brian Davison Brian Davison's Every Which Way,"{'album_type': 'album', 'artists': [{'external..."
9998,Theatre Of Hate,Westworld,Theatre Of Hate Westworld,None
9999,The Black Crowes,Three Snakes And One Charm,The Black Crowes Three Snakes And One Charm,"{'album_type': 'album', 'artists': [{'external..."
10000,Mat Zo,Damage Control,Mat Zo Damage Control,"{'album_type': 'album', 'artists': [{'external..."


In [413]:
best_albums_dataset["album_name"] = best_albums_dataset["album"].map(get_album_name)

In [415]:
best_albums_dataset["album_cover"] = best_albums_dataset["album"].map(get_album_image)

In [418]:
best_albums_dataset["album_id"] = best_albums_dataset["album"].map(get_album_id)

In [420]:
del best_albums_dataset["album_artist"]

In [424]:
del best_albums_dataset["album"]

In [428]:
best_album_dataset = best_albums_dataset[['artists', 'album_cover', 'album_name', 'album_id']]

In [431]:
album_dataset = pd.concat([album_dataset, best_album_dataset], ignore_index=True)

In [449]:
album_dataset = album_dataset.dropna(axis=0, subset=['album_id'])

In [461]:
album_dataset.drop_duplicates(subset=['album_id'], inplace=True)

<ipython-input-461-3135ae331221>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  album_dataset.drop_duplicates(subset=['album_id'], inplace=True)


In [463]:
album_dataset["album_id"].duplicated().any()

False

In [465]:
album_dataset.head(50)

,artists,album_cover,album_name,album_id
0,'Mamie Smith',https://i.scdn.co/image/ab67616d000048510057f5...,Mamie Smith Vol. 1 (1920-1921),5R4HBYPhpEXGLXeyjWFbmT
1,"""Screamin' Jay Hawkins""",https://i.scdn.co/image/ab67616d00004851c8a136...,At Home with Screamin' Jay Hawkins,2lKtE0pl4RtMD2bUwKzdcz
3,'Oscar Velazquez',https://i.scdn.co/image/ab67616d00004851fd2df2...,This Is True House Music,63wFVCPZxdaFL9WKXbIQ5m
4,'Mixe',https://i.scdn.co/image/ab67616d00004851832333...,Mixed Emotions,2iSs4IZZHQ5lrFIbUqC1NX
8,'Francisco Canaro',https://i.scdn.co/image/ab67616d00004851df533f...,Tango Collection,5ib9pkP4jctjCGnZpFQopo
9,'Meetya',https://i.scdn.co/image/ab67616d00004851df2d5c...,Meet Again,1tpjTLdjuiLlokIPtN9faZ
10,'Dorville',https://i.scdn.co/image/ab67616d00004851e9108e...,Doraville,11A5KSBamOsVcMu69sgwzr
12,'Ka Koula',https://i.scdn.co/image/ab67616d00004851f24617...,"Fiddle & Banjo: Tunes from the North, Songs fr...",0Ryhdz1toYz6w1jQcCZu8j
13,'Justrock',https://i.scdn.co/image/ab67616d00004851c04686...,Just Rock,0xwUpEA2QxWbOyxF0ZQkpk
16,'Giorgos Katsaros',https://i.scdn.co/image/ab67616d000048514a2288...,Kathe Limani Kai Kaymos,5ckm0ODsruXhNgeDv1MNRA


In [466]:
album_dataset.to_csv("album_dataset")

In [473]:
album_dataset.describe()

,artists,album_cover,album_name,album_id
count,9643,9641,9644,9644
unique,4695,9641,9522,9644
top,The Rolling Stones,https://i.scdn.co/image/ab67616d000048512a08a3...,Greatest Hits,1dxMK0PJZ9RyarNPXQQeeP
freq,30,1,15,1


In [477]:
pwd

'/Users/wesleyhouse/code/WesH0use/crate_scanner/notebooks'

In [478]:
ls

album_dataset         data.csv              test.jpg
best_albums.csv       spotify_client.ipynb  test_album_df.csv


In [481]:
ls ../crate_scanner

__init__.py  data/


In [482]:
ls -a ..

./                .git/             Makefile          requirements.txt
../               .github/          README.md         scripts/
.DS_Store         .gitignore        crate_scanner/    setup.py
.env              MANIFEST.in       notebooks/        tests/


In [485]:
cat ../.gitignore

*.egg-info
.coverage
.ipynb_checkpoints
.DS_Store
raw_data/
**/__pycache__/
.env


In [486]:
second_album_df

,artists,album_cover,album_name,album_id
0,'Real McCoy',https://i.scdn.co/image/ab67616d0000485126b650...,Another Night,0m4EUHpp4mVnV8AWHT7T7x
1,'DJ Quik',https://i.scdn.co/image/ab67616d000048516413c9...,Quik Is The Name,6qzvk2A6wQpy1f69hrWj3B
2,'Chalino Sanchez',https://i.scdn.co/image/ab67616d000048511243c5...,Nieves de Enero,69CnGWLFrxlYAl5ku4q5XV
3,'No Doubt',https://i.scdn.co/image/ab67616d000048511f0f01...,No Doubt,5Zm0jwZc5J4jWk9rLmABKO
4,'Trisha Yearwood',https://i.scdn.co/image/ab67616d00004851a8635c...,Trisha Yearwood,6US5XlX7w7nL9pRE0F5jeI
...,...,...,...,...
995,'Backstreet Boys',https://i.scdn.co/image/ab67616d00004851dafd4b...,Backstreet Boys,0wvQovgaVU99eqw8n3g22S
996,'Chet Atkins and his Galloping Guitar',None,None,None
997,'Alice DJ',https://i.scdn.co/image/ab67616d00004851d67448...,Who Needs Guitars Anyway?,5oPKlo7IBFXlh12tqDVoAU
998,'Chet Atkins and his Guitar Pickers',None,None,None


In [543]:
tester_df = pd.read_csv('data/best_albums.csv', skiprows=range(1,10))

In [544]:
tester_df.columns

Index(['Rank', 'Title', 'Band', 'Country', 'Year', 'Compilation', 'Live',
       'YearRank', 'DecadeRank', 'OverallRank', 'RankScore', 'AvgRating',
       'NumRatings', 'AlbumID', 'Notes', 'Buy'],
      dtype='object')

In [540]:
tester_df.head()

,Rank,Title,Band,Country,Year,Compilation,Live,YearRank,DecadeRank,OverallRank,RankScore,AvgRating,NumRatings,AlbumID,Notes,Buy
0,1,OK Computer,Radiohead,uk,1997.0,No,No,1.0,1.0,1.0,103160.0,91.853363,5023.0,227.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."
1,2,The Dark Side Of The Moon,Pink Floyd,uk,1973.0,No,No,1.0,1.0,2.0,95111.0,92.282722,4767.0,203.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."
2,3,Abbey Road,The Beatles,uk,1969.0,No,No,1.0,1.0,3.0,82871.0,91.642052,4111.0,24.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."
3,4,Revolver,The Beatles,uk,1966.0,No,No,1.0,2.0,4.0,75445.0,91.192093,4091.0,28.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."
4,5,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,uk,1967.0,No,No,1.0,3.0,5.0,69902.0,90.482063,4116.0,30.0,NaN,"=HYPERLINK(""""https://www.besteveralbums.com/bu..."


## Recommender Dataset

In [498]:
song_df = pd.read_csv('data/data.csv', skiprows=(1,15000), nrows=1000)

In [499]:
song_df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
1,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
2,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
3,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920
4,0.996000,['Mamie Smith & Her Jazz Hounds'],0.424,198627,0.245,0,3HnrHGLE9u2MjHtdobfWl9,0.799000,5,0.2350,-11.470,1,Crazy Blues - 78rpm Version,9,1920,0.0397,103.870,0.4770,1920


In [500]:
song_df.columns

Index(['acousticness', 'artists', 'danceability', 'duration_ms', 'energy',
       'explicit', 'id', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'name', 'popularity', 'release_date', 'speechiness', 'tempo',
       'valence', 'year'],
      dtype='object')

In [504]:
## Clean artist column
song_df["artists"] = song_df["artists"].map(clean_artists_col)

In [505]:
song_df

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.643000,"""Screamin' Jay Hawkins""",0.852,150200,0.5170,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
1,0.993000,'Mamie Smith',0.647,163827,0.1860,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
2,0.000173,'Oscar Velazquez',0.730,422087,0.7980,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
3,0.295000,'Mixe',0.704,165224,0.7070,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920
4,0.996000,'Mamie Smith & Her Jazz Hounds',0.424,198627,0.2450,0,3HnrHGLE9u2MjHtdobfWl9,0.799000,5,0.2350,-11.470,1,Crazy Blues - 78rpm Version,9,1920,0.0397,103.870,0.4770,1920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.967000,'Ethel Waters',0.580,187133,0.2250,0,23furE1azaxq02V6iHZHMt,0.000000,0,0.1180,-16.784,1,Come Up And See Me Sometime,2,1925,0.3190,96.869,0.5020,1925
996,0.996000,'Louis Armstrong & His Hot Five',0.686,181600,0.1270,0,29735iAeYhkpYnVAURWpSa,0.372000,7,0.1120,-17.711,1,Heebie Jeebies,2,1925,0.0818,156.359,0.5920,1925
997,0.992000,'Yvonne George',0.553,170173,0.0903,0,2AbVQuMyvNhUZEsNAQzp8V,0.000000,3,0.2590,-14.881,1,J'ai Pas Su Y Faire,0,1925,0.1010,77.049,0.3790,1925
998,0.957000,'Tommy Dorsey',0.775,224573,0.2350,0,2F8CnNSU7pExOSXzsgf2vz,0.000000,0,0.9550,-11.981,1,Oh! Look At Me Now,2,1925,0.1080,120.288,0.6010,1925


In [545]:
spotify.get_track_by_id("5DlCyqLyX2AOVDTjjkDZ8x")["album"]["id"]

'6os2Mv58OYnQClPf7B9E1s'

In [550]:
song_df["album_id"] = song_df["id"].map(get_album_by_track_id)

In [623]:
album_large_images_v1 = pd.read_csv("/Users/wesleyhouse/code/WesH0use/crate_scanner/crate_scanner/data/full_albums.csv")

In [625]:
album_large_v2 = pd.read_csv("/Users/wesleyhouse/code/WesH0use/crate_scanner/crate_scanner/data/full_albums.csv")

In [641]:
type(album_large_v2["album"])

pandas.core.series.Series

In [648]:
album_large_v2

,artists,album,album_cover,album_name,album_id
0,'Tommy Dorsey',"{'album_type': 'album', 'artists': [{'external...",https://i.scdn.co/image/ab67616d0000b2739675ba...,The Essential Frank Sinatra with the Tommy Dor...,6mcP2ZPyrpMIOrXvSDNQ3g
1,'Francisco Canaro',"{'album_type': 'album', 'artists': [{'external...",https://i.scdn.co/image/ab67616d0000b273df533f...,Tango Collection,5ib9pkP4jctjCGnZpFQopo
2,'Georges Thill',"{'album_type': 'album', 'artists': [{'external...",https://i.scdn.co/image/ab67616d0000b273824a11...,Lebendige Vergangenheit - Georges Thill,3P0msTUeYLGSJ8KXRBL7x8
3,'Ignacio Corsini',"{'album_type': 'album', 'artists': [{'external...",https://i.scdn.co/image/ab67616d0000b273a0640f...,La Pulpera De Santa Lucía,030VxnKCdpPGTvx7s2oMxG
4,'Cliff Edwards',"{'album_type': 'album', 'artists': [{'external...",https://i.scdn.co/image/ab67616d0000b2739b06b3...,The Vintage Recordings of Cliff Edwards (Ukule...,0MFGfJBbZ58E7CIuiUQtZL
...,...,...,...,...,...
9464,OFF!,"{'album_type': 'album', 'artists': [{'external...",https://i.scdn.co/image/ab67616d0000b2734a7011...,First Four EPs,4RbG8CVXYo3in9d1cVAZnt
9465,Brian Davison,"{'album_type': 'album', 'artists': [{'external...",https://i.scdn.co/image/ab67616d0000b2736943b2...,Brian Davison's Every Which Way (2020 Remaster),38t7PKD84dphZNNSLKxg34
9466,The Black Crowes,"{'album_type': 'album', 'artists': [{'external...",https://i.scdn.co/image/ab67616d0000b273db1499...,Three Snakes And One Charm,5KEqZXp6Aw2LF6J5bjelbf
9467,Mat Zo,"{'album_type': 'album', 'artists': [{'external...",https://i.scdn.co/image/ab67616d0000b2732dcfc8...,Damage Control,5Nrx0GEDuLwrQcEvb1IM8l


In [649]:
large_album_image_df = album_large_v2[["artists", "album_cover", "album_name", "album_id"]]

In [653]:
large_album_image_df["album_cover"][100]

'https://i.scdn.co/image/ab67616d0000b273cc7a31a6a403c181a25a91c9'

In [656]:
large_album_image_df.to_csv("large_album_image_df.csv")